In [2]:
import google.generativeai as genai
import google.ai.generativelanguage as glm
import os
from dotenv import load_dotenv

load_dotenv()

genai.configure(api_key=os.environ.get('GOOGLE_API_KEY'))

In [115]:
# 2단계 함수 호출 구현하기
prod_database = {
    "갤럭시 S24": {"재고": 3, "가격": 1700000},
    "갤럭시 S23": {"재고": 5, "가격": 1300000},
    "갤럭시 S22": {"재고": 3, "가격": 1100000},
}

from typing import Tuple


def product_available(product_name: str) -> Tuple[bool, str]:
    """check the product is available or not

    Args:
        product_name: 제품명
    """
    if product_name in prod_database:
        if prod_database[product_name]["재고"] > 0:
            return True, f"현재 {product_name} 재고가 있습니다."
    return False, f"현재 {product_name} 재고는 현재 없습니다."


def product_price(product_name: str) -> Tuple[bool, int]:
    """check the product price

    Args:
        product_name: 제품명
    """
    if product_name in prod_database:
        return True, prod_database[product_name]["가격"]
    return False, None


def place_order(product_name: str, address: str) -> Tuple[bool, str]:
    """제품 주문결과를 반환한다.
    Args:
        product_name: 제품명
        address: 배송지
    """
    if product_available(product_name):
        prod_database[product_name]["재고"] -= 1
        return True, "주문 완료"
    else:
        return False, "재고 부족으로 주문 불가"


function_repoistory = {
    "product_available": product_available,
    "product_price": product_price,
    "place_order": place_order
}


def correct_response(response):
    part = response.candidates[0].content.parts[0]
    if part.function_call:
        for k, v in part.function_call.args.items():
            byte_v = bytes(v, "utf-8").decode("unicode_escape")
            corrected_v = bytes(byte_v, "latin1").decode("utf-8")
            part.function_call.args.update({k: corrected_v})


def make_fc_history(chat_session, part, answer):
    content = glm.Content(parts=[part], role="user")
    chat_session.history.append(content)

    response = glm.GenerateContentResponse({
        "candidates": [{"content": {"role": "model", "parts": [{"text": answer}]}}]
    })
    chat_session.history.append(response.candidates[0].content)
    return response

In [116]:
model = genai.GenerativeModel(
    model_name="gemini-2.0-flash",
    tools=function_repoistory.values()
)

chat_session = model.start_chat(history=[])
queries = ["갤럭시 S24 판매 중인가요?", "갤럭시 S24의 가격은 어떻게 되나요?", "서울시 종로구 종로1가 1번지로 갤럭시 S24를 1개 배송부탁드립니다"]

for query in queries:
    print(f"\n1. 사용자: {query}")
    #response = chat_session.send_message(query)
    print(f"2. history:\n{chat_session.history}")
    if query == "가격은 어떻게 되나요?":
        print(chat_session.send_message(query))
    else:
        response = chat_session.send_message(query)

    correct_response(response)
    part = response.candidates[0].content.parts[0]
    if part.function_call:
        function_call = part.function_call
        function_name = function_call.name
        function_args = {k: v for k, v in function_call.args.items()}
        print(function_repoistory[function_name](**function_args))
        is_required_3_4, function_result = function_repoistory[function_name](**function_args)

        if is_required_3_4 == False:
            print(f'해당 상품의 재고가 없으므로, 나중에 다시 연락주시기 바랍니다.')
            break

        part = glm.Part(
            function_response=glm.FunctionResponse(
                name=function_name,
                response={"content": function_result, },
            )
        )

        print(f'3. part detail : {part.function_response}')
        if is_required_3_4:
            response = chat_session.send_message(part)
        else:
            response = make_fc_history(chat_session, part, function_result)

        print(f"4. 모델: {response.candidates[0].content.parts[0].text}")


1. 사용자: 갤럭시 S24 판매 중인가요?
2. history:
[]
(True, '현재 갤럭시 S24 재고가 있습니다.')
3. part detail : name: "product_available"
response {
  fields {
    key: "content"
    value {
      string_value: "\355\230\204\354\236\254 \352\260\244\353\237\255\354\213\234 S24 \354\236\254\352\263\240\352\260\200 \354\236\210\354\212\265\353\213\210\353\213\244."
    }
  }
}
4. 모델: 네, 갤럭시 S24 판매 중입니다.


1. 사용자: 갤럭시 S24의 가격은 어떻게 되나요?
2. history:
[parts {
  text: "\352\260\244\353\237\255\354\213\234 S24 \355\214\220\353\247\244 \354\244\221\354\235\270\352\260\200\354\232\224?"
}
role: "user"
, parts {
  function_call {
    name: "product_available"
    args {
      fields {
        key: "product_name"
        value {
          string_value: "\352\260\244\353\237\255\354\213\234 S24"
        }
      }
    }
  }
}
role: "model"
, parts {
  function_response {
    name: "product_available"
    response {
      fields {
        key: "content"
        value {
          string_value: "\355\230\204\354\236\254 \352

In [117]:
print(f"history:\n{chat_session.history}")

history:
[parts {
  text: "\352\260\244\353\237\255\354\213\234 S24 \355\214\220\353\247\244 \354\244\221\354\235\270\352\260\200\354\232\224?"
}
role: "user"
, parts {
  function_call {
    name: "product_available"
    args {
      fields {
        key: "product_name"
        value {
          string_value: "\352\260\244\353\237\255\354\213\234 S24"
        }
      }
    }
  }
}
role: "model"
, parts {
  function_response {
    name: "product_available"
    response {
      fields {
        key: "content"
        value {
          string_value: "\355\230\204\354\236\254 \352\260\244\353\237\255\354\213\234 S24 \354\236\254\352\263\240\352\260\200 \354\236\210\354\212\265\353\213\210\353\213\244."
        }
      }
    }
  }
}
role: "user"
, parts {
  text: "\353\204\244, \352\260\244\353\237\255\354\213\234 S24 \355\214\220\353\247\244 \354\244\221\354\236\205\353\213\210\353\213\244.\n"
}
role: "model"
, parts {
  text: "\352\260\244\353\237\255\354\213\234 S24\354\235\230 \352\260\

In [118]:
chat_session

ChatSession(
    model=genai.GenerativeModel(
        model_name='models/gemini-2.0-flash',
        generation_config={},
        safety_settings={},
        tools=<google.generativeai.types.content_types.FunctionLibrary object at 0x10be65b40>,
        system_instruction=None,
        cached_content=None
    ),
    history=[protos.Content({'parts': [{'text': '갤럭시 S24 판매 중인가요?'}], 'role': 'user'}), protos.Content({'parts': [{'function_call': {'args': {'product_name': '갤럭시 S24'}, 'name': 'product_available'}}], 'role': 'model'}), protos.Content({'parts': [{'function_response': {'name': 'product_available', 'response': {'content': '현재 갤럭시 S24 재고가 있습니다.'}}}], 'role': 'user'}), protos.Content({'parts': [{'text': '네, 갤럭시 S24 판매 중입니다.\n'}], 'role': 'model'}), protos.Content({'parts': [{'text': '갤럭시 S24의 가격은 어떻게 되나요?'}], 'role': 'user'}), protos.Content({'parts': [{'function_call': {'args': {'product_name': '갤럭시 S24'}, 'name': 'product_price'}}, {'text': '\n'}], 'role': 'model'}), protos.Conte